#### Not working - need "agent id"

In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
url = "https://www.sec.gov/Archives/edgar/data/1041673/0001193125-20-135785.txt"
req = requests.get(url)
soup = BeautifulSoup(req.text, "html.parser")
print(soup.title)

In [ ]:
for link in soup.find_all('a'):
    print(link.get('href'))

#### Iterate over each filling and clean html data to readible text

In [ ]:
import os
from bs4 import BeautifulSoup
import re
from utils import load_pkl

In [ ]:
# Load file containing all available CIK on CRSP databse
cik_available_pkl = load_pkl('cik_available_pkl.pkl')

In [ ]:
# Clean fillings by removing html tags, images and some characters
def remove_tags(html):
    search_filling = re.search("(?s)(?m)<TEXT>.*?(</TEXT>)", html)
    try:
        data_processed = search_filling.group(0)

        # delete formatting text used to identify specific section as its not relevant
        data_processed = re.sub(pattern="((?i)<TYPE>).*?(?=<)", repl='', string=data_processed)

        # Five more formatting tags are deleted
        data_processed = re.sub(pattern="((?i)<SEQUENCE>).*?(?=<)", repl='', string=data_processed)
        data_processed = re.sub(pattern="((?i)<FILENAME>).*?(?=<)", repl='', string=data_processed)
        data_processed = re.sub(pattern="((?i)<DESCRIPTION>).*?(?=<)", repl='', string=data_processed)
        data_processed = re.sub(pattern="(?s)(?i)<head>.*?</head>", repl='', string=data_processed)
        data_processed = re.sub(pattern="(?s)(?i)<(table).*?(</table>)", repl='', string=data_processed)

        # Removes all HTML tags
        data_processed = re.sub(pattern="(?s)<.*?>", repl=" ", string=data_processed, count=0)

        # Replaces all Unicode strings
        data_processed = re.sub(pattern="&(.{2,6});", repl=" ", string=data_processed, count=0)

        data_processed = re.sub(u'[\t\n\xa0]', ' ', data_processed)

        # Replaces multiple spaces with a single space
        data_processed = re.sub(pattern="(?s) +", repl=" ", string=data_processed, count=0)
        return data_processed
    except :
        return None

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
DS_STORE = '.DS_Store'

fillings_cik_cleaned=[]
try:
    fillings_cik_cleaned = os.listdir('data/sec-edgar-filings-cleaned')
    if DS_STORE in fillings_cik_cleaned : fillings_cik_cleaned.remove(DS_STORE)
    nb_elem = len(fillings_cik_cleaned)
except:
    nb_elem = 0
    
print(nb_elem)

In [ ]:
fillings_cik = os.listdir('data/sec-edgar-filings')
DS_STORE = '.DS_Store'

if DS_STORE in fillings_cik : fillings_cik.remove(DS_STORE)
n = len(fillings_cik)
for i, cik in enumerate(sorted(fillings_cik)) : # all fillings
    if cik in fillings_cik_cleaned or cik not in cik_available_pkl:
        continue
    print(f"{cik} --- {i}/{n} ({i/n*100:.4}%)", end='\r')
    
    path_cik = 'data/sec-edgar-filings/'+str(cik)
    report_type = os.listdir(path_cik)
    if DS_STORE in report_type : report_type.remove(DS_STORE)
    
    for rt in report_type : # N-CSR and N-CSRS
        path_report_type = path_cik+'/'+str(rt)
        report_folder = os.listdir(path_report_type)
        if DS_STORE in report_folder : report_folder.remove(DS_STORE)
        
        for rf in report_folder: # report identity
            path_report_folder = path_report_type+'/'+rf
            filling = os.listdir(path_report_folder)
            
            for file in filling: # file submission text
                path_file = path_report_folder+'/'+str(file)
                with open(path_file, 'r') as f:
                    # Prepare new directory where to save the clean / parsed version of the report
                    path_clean = "data/sec-edgar-filings-cleaned/"+'/'.join(path_file.split('/')[2:-1])
                    if not os.path.exists(path_clean):
                         os.makedirs(path_clean)
                    text = remove_tags(f.read())
                    if text is not None:
                        # Save in txt file the new SEC report filling
                        with open(path_clean+"/full-submission.txt", 'w') as f:
                            f.write(text)

#### Find benchmarks candidates

In [ ]:
with open('beautifulsoup.txt', 'w') as f:
    f.write(text)

In [ ]:
import numpy as np
words = text.split()

indices = [ i for i, w in enumerate(words) if w.lower() == 'benchmark']

for i in indices:
    potential = words[i-15:i+15]
    potential = [p for p in potential if not p.islower() or p == 'benchmark']
    print(' '.join(words[i-15:i+15]), end='\n\n')

In [ ]:
  # Print the extracted data
print(remove_tags(req.content))

#### Test with one file

In [ ]:
path = "data/sec-edgar-filings/0001553197/N-CSR/0000051931-17-000292/full-submission.txt"
with open(path, 'r') as f:
     text = remove_tags(f.read())

In [ ]:
path = "data/sec-edgar-filings/0001553197/N-CSR/0000051931-17-000292/cleaned_draft.txt"
with open(path, 'w') as f:
    if text is not None:
        f.write(text)